# Task description
Use transformer to classify the speakers of given features.

In [1]:
!pip install gdown

In [2]:

!unzip data.zip

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=169ghKW0xyLbOuMa7rrRUj6XN3QrA-FI0
From (redirected): https://drive.google.com/uc?id=169ghKW0xyLbOuMa7rrRUj6XN3QrA-FI0&confirm=t&uuid=684e1a66-7812-4c11-a064-038b65a0e62e
To: /kaggle/working/data.zip
100%|██████████████████████████████████████| 7.54G/7.54G [08:30<00:00, 14.8MB/s]
Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/test_id.csv        
   creating: data/test/
  inflating: __MACOSX/data/._test    
   creating: data/train/
  inflating: data/train_label.csv    
  inflating: data/test/uttr3300.pt   
  inflating: __MACOSX/data/test/._uttr3300.pt  
  inflating: data/test/uttr3091.pt   
  inflating: __MACOSX/data/test/._uttr3091.pt  
  inflating: data/test/uttr120

## Dataset

- 600 speakers, preprocess the raw waveforms into mel-spectrograms.


For efficiency, we segment the mel-spectrograms into segments in the training step.

In [3]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import pandas as pd

"""This custom dataset class is designed to load mel-spectrograms
and corresponding speaker labels from a directory structure
and provide these data samples for training a neural network
for speaker identification."""

# defines a custom PyTorch dataset or class definition
class myDataset(Dataset): #inherits from PyTorch's Dataset class
  def __init__(self, data_dir, segment_len=128): #constructor method
    self.data_dir = data_dir
    self.segment_len = segment_len #as an instance variable

    # Get the total number of speaker.
    self.data = pd.read_csv(os.path.join(data_dir,'train_label.csv'))
    self.speaker_num = len(self.data.speaker_id.unique())

  def __len__(self): #method
    return len(self.data)

  def __getitem__(self, index): #required 4 all PyTorch datasets
    feat_path, speaker = self.data.iloc[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, 'train', feat_path))

    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long integer for compatibility with computing loss later.
    #converts speakerID to PyTorch FloatTensor and then to long
    speaker = torch.FloatTensor([speaker]).long()
    #returns mel-spectrogram and speakerID as a tuple
    return mel, speaker

  def get_speaker_number(self):
    return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [4]:
import torch
from torch.utils.data import DataLoader, random_split #class
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data.
  This function takes a batch of data as input.
  In this case, a batch is a list of tuples where
  each tuple contains a mel-spectrogram (mel) and
  its corresponding speaker ID (speaker)."""

  #unzips the batch into 2 separate lists
  mel, speaker = zip(*batch)
  """Because we train the model batch by batch,
  we need to pad the features in the same batch to make their
  lengths the same."""
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  #returns padded mel-spectrograms
  #speakerIDs as long tensors
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  #creates an instance of myDataset class
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  #creates list of 2 integers
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num


# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [5]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# #Classifier= NN model
# class Classifier(nn.Module):#inherits from nn.Module(base class for all PyTorch models)
#   #constructor initializes the components and paras of model
#   def __init__(self, d_model=80, n_spks=600, dropout=0.1):

#     super().__init__()# It calls the constructor of the parent class (nn.Module) to properly initialize it.

#     # linear layer that project the input features into d_model(higher dim space)
#     #initial feature transformation
#     self.prenet = nn.Linear(40, d_model)

#     # TODO:
#     #  1. Use multiple Transformer Encoder layers: nn.TransformerEncoder()
#     #  2. Change Transformer to Conformer. https://arxiv.org/abs/2005.08100

#     #defines T encoder layer
#     self.encoder_layer = nn.TransformerEncoderLayer(
#       d_model=d_model, dim_feedforward=256, nhead=2
#     )
#     # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

#     # Project the the dimension of features from d_model into speaker nums.
#     #sequential module with 2 linear layers + ReLU
#     #projects the output of T encoder into final output space-> # of speaker classes
#     self.pred_layer = nn.Sequential(
#       nn.Linear(d_model, d_model),
#       nn.ReLU(),
#       nn.Linear(d_model, n_spks),
#     )

#   #defines the forward pass of the model
#   def forward(self, mels):
#     """
#     args:
#       mels: (batch size, length, 40)
#     return:
#       out: (batch size, n_spks)
#     """
#     # out: (batch size, length, d_model)

#     """This line projects the input mel-spectrogram features (mels)
#     into the d_model-dimensional space using the self.prenet
#     linear layer."""
#     out = self.prenet(mels)

#     # out: (length, batch size, d_model)
#     """This rearranges the dimensions of out to be in the shape
#     expected by the Transformer encoder,
#     which is (length, batch size, d_model)"""
#     out = out.permute(1, 0, 2)

#     # The encoder layer expect features in the shape of (length, batch size, d_model).
#     """applies the TransformerEncoderLayer to the input features.
#     The out tensor now represents the processed features"""
#     out = self.encoder_layer(out)

#     # out: (batch size, length, d_model)
#     #rearranges the dimensions back to (batch size, length, d_model)
#     out = out.transpose(0, 1)


#     """#computes mean along length dimension resulting
#     in a feature representation of shape (batch size, d_model)
#     It is a form of pooling over the time dimension
#     to obtain a fixed length representation for each input sequence"""
#     stats = out.mean(dim=1)

#     # out: (batch, n_spks)
#     """Finally, the mean-pooled features are passed through
#     the self.pred_layer, which projects them into the output space
#     with n_spks dimensions, representing the speaker classes"""
#     out = self.pred_layer(stats)
#     return out


In [12]:
import torch
import torch.nn as nn

class ConformerEncoderLayer(nn.Module):
    def __init__(self, d_model=80, nhead=4, dim_feedforward=512, dropout=0.4):
        super().__init__()

        # Multi-Head Self-Attention
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)

        # # Batch Normalization before activation functions
        # self.norm1 = nn.BatchNorm1d(d_model)
        # self.norm2 = nn.BatchNorm1d(d_model)

        # Feedforward Neural Network
        self.feedforward = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim_feedforward, d_model),
        )

        # Layer Normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # Position-wise Feedforward Networks
        self.positionwise_ffn = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim_feedforward, d_model),
        )

    def forward(self, x):
        # Multi-Head Self-Attention followed by residual connection and layer normalization
        attn_output, _ = self.self_attn(x, x, x)
        x = x + attn_output
        x = self.norm1(x)

        # Position-wise Feedforward Network followed by residual connection and layer normalization
        ffn_output = self.feedforward(x)
        x = x + ffn_output
        x = self.norm2(x)

        return x

class ConformerEncoder(nn.Module):
    def __init__(self, d_model=80, num_layers=3, nhead=4, dim_feedforward=512, dropout=0.3):
        super().__init__()

        # Stack multiple ConformerEncoderLayers
        self.layers = nn.ModuleList([ConformerEncoderLayer(d_model, nhead, dim_feedforward, dropout) for _ in range(num_layers)])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class Classifier(nn.Module):
    def __init__(self, d_model=80, n_spks=600, num_encoder_layers=3, nhead=4, dim_feedforward=512, dropout=0.3):
        super().__init__()

        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)

        # Conformer Encoder
        self.encoder = ConformerEncoder(d_model, num_layers=num_encoder_layers, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)

        # Project the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Linear(d_model, n_spks),
        )

    def forward(self, mels):
        # Project the input features
        out = self.prenet(mels)

        # Transpose for compatibility with ConformerEncoder
        out = out.permute(1, 0, 2)

        # Pass through the Conformer Encoder
        out = self.encoder(out)

        # Transpose back
        out = out.transpose(0, 1)

        # Mean pooling
        stats = out.mean(dim=1)

        # Final prediction layer
        out = self.pred_layer(stats)

        return out


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [13]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)


# Model Function
- Model forward function.

In [14]:
import torch


def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [15]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)


# Main function

In [16]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
  """arguments"""
  config = {
    "data_dir": "./data",
    "save_path": "model.ckpt",
    "batch_size": 512,
    "n_workers": 8,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 70000,
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)

  model = Classifier(n_spks=speaker_num).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    # Log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # Do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())


[Info]: Use cuda now!


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[Info]: Finish loading data!
[Info]: Finish creating model!




Train:   0% 0/2000 [00:00<?, ? step/s]

Train:   0% 1/2000 [00:01<49:54,  1.50s/ step]

Train:   0% 1/2000 [00:01<49:54,  1.50s/ step, accuracy=0.00, loss=6.41, step=1]

Train:   0% 2/2000 [00:01<26:57,  1.24 step/s, accuracy=0.00, loss=6.41, step=1]

Train:   0% 2/2000 [00:01<26:57,  1.24 step/s, accuracy=0.00, loss=6.42, step=2]

Train:   0% 3/2000 [00:01<26:56,  1.24 step/s, accuracy=0.00, loss=6.41, step=3]

Train:   0% 4/2000 [00:02<12:07,  2.75 step/s, accuracy=0.00, loss=6.41, step=3]

Train:   0% 4/2000 [00:02<12:07,  2.75 step/s, accuracy=0.00, loss=6.41, step=4]

Train:   0% 5/2000 [00:02<12:06,  2.75 step/s, accuracy=0.00, loss=6.40, step=5]

Train:   0% 6/2000 [00:02<07:58,  4.17 step/s, accuracy=0.00, loss=6.40, step=5]

Train:   0% 6/2000 [00:02<07:58,  4.17 step/s, accuracy=0.00, loss=6.40, step=6]

Train:   0% 7/2000 [00:02<07:57,  4.17 step/s, accuracy=0.00, loss=6.42, step=7]

Train:   0% 8/2000 [00:02<06:04,  5.46 step/s, accuracy=0.00, loss=6.42, step=7]

Train:  

Step 10000, best model saved. (accuracy=0.7830)




Train:   0% 3/2000 [00:00<02:31, 13.21 step/s, accuracy=0.88, loss=0.42, step=1e+4]

Train:   0% 4/2000 [00:00<02:54, 11.43 step/s, accuracy=0.88, loss=0.42, step=1e+4]

Train:   0% 4/2000 [00:00<02:54, 11.43 step/s, accuracy=0.88, loss=0.46, step=1e+4]

Train:   0% 5/2000 [00:00<02:54, 11.43 step/s, accuracy=0.86, loss=0.48, step=1e+4]

Train:   0% 6/2000 [00:00<03:08, 10.59 step/s, accuracy=0.86, loss=0.48, step=1e+4]

Train:   0% 6/2000 [00:00<03:08, 10.59 step/s, accuracy=0.88, loss=0.41, step=1e+4]

Train:   0% 7/2000 [00:00<03:08, 10.59 step/s, accuracy=0.88, loss=0.47, step=1e+4]

Train:   0% 8/2000 [00:00<03:10, 10.48 step/s, accuracy=0.88, loss=0.47, step=1e+4]

Train:   0% 8/2000 [00:00<03:10, 10.48 step/s, accuracy=0.89, loss=0.40, step=1e+4]

Train:   0% 9/2000 [00:00<03:09, 10.48 step/s, accuracy=0.92, loss=0.33, step=1e+4]

Train:   0% 10/2000 [00:00<03:14, 10.23 step/s, accuracy=0.92, loss=0.33, step=1e+4]

Train:   0% 10/2000 [00:00<03:14, 10.23 step/s, accuracy=0.88,

Step 40000, best model saved. (accuracy=0.8621)


Train: 100% 2000/2000 [06:51<00:00,  4.86 step/s, accuracy=0.97, loss=0.07, step=42000]
Valid:  98% 6144/6250 [00:04<00:00, 1245.56 uttr/s, accuracy=0.85, loss=0.86]
Train: 100% 2000/2000 [06:50<00:00,  4.88 step/s, accuracy=0.98, loss=0.08, step=44000]
Valid:  98% 6144/6250 [00:04<00:00, 1379.23 uttr/s, accuracy=0.86, loss=0.83]
Train: 100% 2000/2000 [06:49<00:00,  4.89 step/s, accuracy=0.97, loss=0.08, step=46000]
Valid:  98% 6144/6250 [00:05<00:00, 1224.32 uttr/s, accuracy=0.87, loss=0.84]
Train: 100% 2000/2000 [06:48<00:00,  4.89 step/s, accuracy=0.98, loss=0.07, step=48000]
Valid:  98% 6144/6250 [00:06<00:00, 1018.16 uttr/s, accuracy=0.87, loss=0.83]
Train: 100% 2000/2000 [06:59<00:00,  4.77 step/s, accuracy=1.00, loss=0.01, step=7e+4] 
Valid:  98% 6144/6250 [00:06<00:00, 1023.43 uttr/s, accuracy=0.87, loss=0.86]
Train:   0% 0/2000 [00:00<?, ? step/s]
                                                                                       
Train:   0% 0/2000 [00:00<?, ? step/s]4.71 

Step 70000, best model saved. (accuracy=0.8779)


# Inference

## Dataset of inference

In [17]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    self.data_dir = data_dir
    self.data = pd.read_csv(os.path.join(data_dir,'test_id.csv'))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    feat_path = self.data.feat_path[index]
    mel = torch.load(os.path.join(self.data_dir, 'test', feat_path))

    return feat_path, mel


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)


## Main function of Inference

In [18]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "./data",
    "model_path": "./model.ckpt",
    "output_path": "./output512psimplegelu.csv",
  }

  return config


def main(
  data_dir,
  model_path,
  output_path,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = 600
  model = Classifier(n_spks=speaker_num).to(device)
  model.load_state_dict(torch.load(model_path))
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, pred])

  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())


[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/6944 [00:00<?, ?it/s]